In [1]:
import torch
import matplotlib.pyplot as plt


In [2]:
a = torch.zeros(4, 2)
b = a.chunk(2, dim=0)
print(f'a.shape {a.shape}')


a.shape torch.Size([4, 2])


In [3]:
a.to('cuda:0')
c = torch.zeros((4,2), device = 'cuda:1')
c = a

In [ ]:
import torch.distributed as dist
import os
os.environ['MASTER_ADDR'] = '127.0.0.1'
os.environ['MASTER_PORT'] = '29500'
dist.init_process_group('nccl', rank=0, world_size=2)


K1 = torch.ones(4,2)
K1 = K1.to('cuda:0')
K2 = torch.ones(4,2)
K2 = K2.to('cuda:0')

gather_list = []
output_tensor = []
world_size = dist.get_world_size()
for tensor in [K1, K2]:
    tensor_placeholder = [
        torch.ones_like(tensor) for _ in range(world_size)
    ]
    dist.all_gather(tensor_placeholder, tensor, async_op=False)
    gather_list.append(tensor_placeholder)
for gathered_tensor in gather_list:
    output_tensor.append(torch.cat(gathered_tensor, dim=0))


In [ ]:
import torch
import torch.distributed as dist
import os

os.environ['MASTER_ADDR'] = '127.0.0.1'
os.environ['MASTER_PORT'] = '29500'
dist.init_process_group(backend="nccl",
                        world_size=2,
                        rank=0)
tensor_list = []
for dev_idx in range(torch.cuda.device_count()):
    tensor_list.append(torch.FloatTensor([1]).cuda(dev_idx))

dist.all_reduce_multigpu(tensor_list)

In [3]:
A = torch.randn(32, 2, 512, 16)
B = torch.randn_like(A)
B.transpose(-1, -2).shape
torch.matmul(A, B.transpose(-1, -2)).shape

torch.Size([32, 2, 512, 512])